In [1]:
# Importar las bibliotecas necesarias

import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import pytest
import ipytest
ipytest.autoconfig()  # Configurar ipytest para usar pytest en Jupyter Notebook

In [2]:
# Leer el dataset desde un archivo CSV
dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv") # cambiar la ruta al probar

# Crear un DataFrame a partir del dataset leído
dataset_df = pd.DataFrame(dataset, columns=dataset.columns)

# Convertir la columna "Grade" a tipo categórico
dataset_df["Grade"] = dataset_df["Grade"].astype('category')

In [3]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Carga el dataset y realiza la división en conjuntos de entrenamiento y prueba."""
        dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv") # cambiar la ruta al probar
        self.frame = pd.DataFrame(dataset, columns=dataset.columns)
        self.frame["Grade"] = self.frame["Grade"].astype('category')  # Convertir la columna "Grade" a tipo categórico
        
        # Obtener los nombres de las características excluyendo la columna 'Grade'
        feature_names = [col for col in self.frame.columns if col != 'Grade']
        
        # Dividir el dataset en conjuntos de entrenamiento y prueba
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[feature_names], self.frame['Grade'], test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        """Entrenar el modelo usando el algoritmo especificado (por defecto LogisticRegression)."""
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        """Realizar predicciones usando el modelo entrenado."""
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        """Obtener la precisión del modelo en el conjunto de prueba."""
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Método de ejecución para correr el pipeline varias veces."""
        self.load_dataset()
        self.train()

In [4]:
class PipelineWithConfig(SimplePipeline):
    def __init__(self, config):
        # Llamar primero al método __init__ de la clase heredada SimplePipeline.
        super().__init__()
        # Pasamos un objeto de configuración que se usará en el entrenamiento.
        # Esto se conoce como Inyección de Dependencias.
        self.config = config
            
    def train(self, algorithm=LogisticRegression):
        # Inicializamos el clasificador LogisticRegression con los parámetros de la configuración del pipeline.
        self.model = algorithm(solver=self.config.get('solver'),
                               multi_class=self.config.get('multi_class'))
        self.model.fit(self.X_train, self.y_train)

Testeo

In [5]:
@pytest.fixture
def pipeline():
    # Crear una configuración para el modelo
    config = {'solver': 'lbfgs', 'multi_class': 'auto'}
    
    # Crear una instancia del pipeline con la configuración especificada
    pl = PipelineWithConfig(config=config)
    
    # Ejecutar el pipeline (cargar el dataset y entrenar el modelo)
    pl.run_pipeline()
    
    # Devolver la instancia del pipeline
    return pl

In [7]:
%%ipytest

# Definir un conjunto de solvers habilitados para el modelo
ENABLED_MODEL_SOLVERS = {'newton-cg', 'lbfgs'}

def test_pipeline_config(pipeline):
    # Obtener los parámetros de configuración del modelo
    model_params = pipeline.model.get_params()

    # Comparar el solver del modelo con los solvers habilitados
    assert model_params['solver'] in ENABLED_MODEL_SOLVERS
    # assert False, repr(model_params)

.                                                                                            [100%]
1 passed in 0.06s
